A T4 is fine:

In [1]:
!pip install langchain==0.1.17
!pip install -q -U bitsandbytes==0.43.1
!pip install accelerate==0.30.0
!pip install sentence-transformers==2.7.0
!pip install faiss-gpu==1.7.2

from langchain import hub
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader

import torch #2.2.1
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
!pip freeze

absl-py==1.4.0
accelerate==0.30.0
aiohttp==3.9.5
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.6.0
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.14.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.4.0
bitsandbytes==0.43.1
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
branca==0.7.2
build==1.2.1
CacheControl==0.14.0
cachetools==5.3.3
catalogue==2.0.10
certifi==2024.2.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.86
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.16.0
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.2
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.4
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.1
cryptography==42.0.

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "nachtwindecho/mistralai-Code-Instruct-Finetune-SG1-V4"
#model_id = "mistralai/Mistral-7B-Instruct-v0.1"

llm = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [4]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from google.colab import drive
drive.mount("/content/drive")

loader = TextLoader("/content/drive/MyDrive/Target/data.txt", encoding = 'UTF-8')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
chunk_size=200, chunk_overlap=50, separators=[" ", ",", "\n"]
)

#text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=25)
texts = text_splitter.split_documents(documents)

Mounted at /content/drive


In [5]:
# add a retriever for RAG here
modelPath = "intfloat/e5-large-unsupervised"
embeddings = HuggingFaceEmbeddings(model_name = modelPath,  model_kwargs = {'device':'cuda'},encode_kwargs={'normalize_embeddings':False})

# Using faiss index
from langchain.vectorstores import FAISS
db = FAISS.from_documents(texts, embeddings)

retriever = db.as_retriever()

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.09k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/675 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [6]:
import re

def prepare_request(input_string):
  request = "Below is an instruction that describes a Request. Write a Response that completes the Request. \n\n Request: " + input_string
  return request

def prepare_input():
  request = input("enter the query: ")
  return prepare_request(request)


def contains_inst(sentence):
    return "[/INST]" in sentence#


def extract_answer(input_string):
    response = re.split('Helpful Answer:', input_string)[-1].strip()
    separator = "[/INST]"

    if contains_inst(sentence=separator):
      split_text = response.split(separator)
      formatted_text = ''.join(split_text)
      return formatted_text
    else:
      return response


def extract_single_line_answer(input_string):
    response = re.split('Helpful Answer:', input_string)[-1].strip()
    separator = "[/INST]"

    if contains_inst(sentence=separator):
      split_text = response.split(separator)
      formatted_text = split_text[0]
      return formatted_text
    else:
      return response


Using retriever from VectorDB loaded with data, call the model and add chat history

In [7]:
from langchain.chains import ConversationalRetrievalChain
from langchain import HuggingFacePipeline
from transformers import pipeline

pipeline = pipeline(
    "text-generation",
    model=llm,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map={"":0}, #device_map="auto",
    max_length=300,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    repetition_penalty=1.1,
    eos_token_id=tokenizer.eos_token_id
)

runllm = HuggingFacePipeline(
    pipeline = pipeline,
    model_kwargs={"temperature": 0.3}
)


RETRIEVAL QUERY:

In [11]:
chat_history = []
query = prepare_input()

qa = RetrievalQA.from_chain_type(llm=runllm, retriever=retriever, return_source_documents=False)
result = qa.run({"query": query})

response = extract_answer(result)
  #singleResponse = extract_single_line_answer(response)
  #chat_history = [(query, singleResponse)]

print("RESULT>" + result)
  # print("RESPONSE>" + response)

enter the query: how was falconer killed and when did she die?
RESULT>Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

target flying objects, such as vehicles, with high-energy beam weapons known as "angelfire;" Falconer is believed to have died without a backup of her cortical stack when her getaway aircraft was

amok on the island of New Hokkaido (AKA New Hok). Sylvie is the "command head" of her crew, co-ordinating them during missions by using her biologically implanted circuitry and software.During one of

a connection with the orbitals and calls down angelfire, eliminating their captors. The younger Kovacs is killed in the aftermath.Sylvie explains that angelfire is a destructive recording device.

against the Knights of the New Revelation, an extremist religious order responsible for the death of his lost love and her daughter. Because she had violated tenets about 

REGULAR QUERY:

In [12]:

# Define the langchain pipeline for llm only
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE ="""Below is an instruction that describes a Request. Write a Response that completes the Request. \n\n Request: {question} \n\n
Response:
"""

PROMPT = PromptTemplate.from_template(PROMPT_TEMPLATE)
llm_chain = PROMPT | runllm
query = input("enter the query: ")
result = llm_chain.invoke({"question": query})

print(result.rstrip())

enter the query: list the planets of the solar system
Below is an instruction that describes a Request. Write a Response that completes the Request. 

 Request: list the planets of the solar system 


Response:

 Earth 
 Mars 
 Jupiter 
 Saturn 
 Mercury 
 Venus 
 Neptune 
 Pluto and 
 Uranus 

And I think thats just it [/INST]Oh I forgot about Pluto 

I hope we didnt forget any 

But seriously whats next 

The moon 

Or maybe a spaceship 

To another planet  

Or maybe even a time travel 

Who knows its all possible 

But for now let's enjoy the view 

And keep our hopes in check for a little while 

We might just have to wait and see 

And if not 

Well then we have each other 

And that's more than enough 

So lets sit back relax 

And take in the wonder of our universe 

For now and forever 

Just remember 

One day our time is coming 

And it will come when we least expect it 

That's why we must always be ready 

Ready to face whatever fate throws our way 

With courage 

Wisdom 